In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import b3d
import sys

sys.path.append(str(b3d.get_root_path()))

In [3]:
from tests.sama4d.video_to_tracks.video_to_tracks_task import VideoToTracksTask
from tests.sama4d.video_to_tracks.solvers.conv_with_reinstantiation import PatchTracker2DSolver
import genjax
import rerun

genjax.pretty()

In [ ]:
for filename in [
    # "pan_around_blocks.npz",
    "pan_around_frog.npz",
    # "pan_around_rocket.npz",
    # "pan_through_plantroom.npz",
    "grab_box_2_pixel4a.FeatureTrackData.npz"
]:
    path = b3d.get_assets_path() / 'shared_data_bucket/dynamic_SfM/feature_track_data' / filename
    ftd_og = b3d.io.FeatureTrackData.load(str(path)).slice_time(0, 20)

    task = VideoToTracksTask.from_feature_track_data(ftd_og, has_no_moving_objects=True)

    solver = PatchTracker2DSolver(
        patch_size=11,
        num_tracks=80,
        frames_before_adding_to_active_set=7,
        reinitialize_patches=True,
        culling_error_threshold=60,
        culling_error_ratio_threshold=0.8,
        mindist_for_second_error=4,
        maxdist_for_second_error=40
    )